In [12]:
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [13]:
!pip install torch
!pip install transformers
!pip install tqdm

In [14]:
import json
import random
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional

from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from transformers import BertForSequenceClassification, BertTokenizer, BertModel, AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaModel

In [15]:
# List of relation types
keys = ['no_relation', 'per:title', 'org:top_members/employees',
        'org:country_of_headquarters', 'per:parents', 'per:age',
        'per:countries_of_residence', 'per:children', 'org:alternate_names',
        'per:charges', 'per:cities_of_residence', 'per:origin', 'org:founded_by',
        'per:employee_of', 'per:siblings', 'per:alternate_names', 'org:website',
        'per:religion', 'per:stateorprovince_of_death', 'org:parents',
        'org:subsidiaries', 'per:other_family', 'per:stateorprovinces_of_residence',
        'org:members', 'per:cause_of_death', 'org:member_of',
        'org:number_of_employees/members', 'per:country_of_birth',
        'org:shareholders', 'org:stateorprovince_of_headquarters',
        'per:city_of_death', 'per:date_of_birth', 'per:spouse',
        'org:city_of_headquarters', 'per:date_of_death', 'per:schools_attended',
        'org:political/religious_affiliation', 'per:country_of_death',
        'org:founded', 'per:stateorprovince_of_birth', 'per:city_of_birth',
        'org:dissolved']

# Assigning indices to the list elements and storing them in a dictionary
rel2id = {key: idx for idx, key in enumerate(keys)}
id2rel = {idx: key for key, idx in rel2id.items()}

In [16]:
# Model
class BERT_Classifier(nn.Module):
    def __init__(self, label_num):
        super().__init__()
        # Initialize the BERT encoder from pre-trained weights
        self.encoder = BertModel.from_pretrained('bert-base-uncased')
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.1, inplace=False)
        # Fully connected layer for classification
        self.fc = nn.Linear(768, label_num)  # 768 is the hidden size of BERT
        # Cross-entropy loss criterion
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x, attention_mask, label=None):
        # Pass the input through the BERT encoder
        x = self.encoder(x, attention_mask=attention_mask)[0]  # Output is tuple (last_hidden_state, pooler_output), we take the last_hidden_state
        # Take only the first token's output (CLS token)
        x = x[:, 0, :]
        # Apply dropout
        x = self.dropout(x)
        # Pass through the fully connected layer
        x = self.fc(x)
        # If label is not provided, return logits only
        if label is None:
            return None, x
        else:
            # Calculate the cross-entropy loss and return both loss and logits
            return self.criterion(x, label), x

class RoBERTa_Classifier(nn.Module):
    def __init__(self, label_num):
        super().__init__()
        # Initialize the RoBERTa encoder from pre-trained weights
        self.encoder = RobertaModel.from_pretrained("roberta-base")
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.1, inplace=False)
        # Fully connected layer for classification
        self.fc = nn.Linear(768, label_num)  # 768 is the hidden size of RoBERTa
        # Cross-entropy loss criterion
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x, attention_mask, label=None):
        # Pass the input through the RoBERTa encoder
        x = self.encoder(x, attention_mask=attention_mask)[0]  # Output is tuple (last_hidden_state, pooler_output), we take the last_hidden_state
        # Take only the first token's output (CLS token)
        x = x[:, 0, :]
        # Apply dropout
        x = self.dropout(x)
        # Pass through the fully connected layer
        x = self.fc(x)
        # If label is not provided, return logits only
        if label is None:
            return None, x
        else:
            # Calculate the cross-entropy loss and return both loss and logits
            return self.criterion(x, label), x

In [17]:
def predict_relation(model_tpye, ent1, ent2, text, model_path, rel2id, id2rel):
    # Define the device based on CUDA availability
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    labels_num = len(rel2id)

    if model_tpye == "BERT":
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
        model = BERT_Classifier(labels_num).to(device)
    elif model_tpye == "RoBERTa":
        tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
        model = RoBERTa_Classifier(labels_num).to(device)
    else:
        print("Wrong model type!")

    # Load the model state
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()

    # Preparing the inputs for the model
    max_length = 128
    sentence = ent1 + ent2 + text
    indexed_tokens = tokenizer.encode(sentence, add_special_tokens=True, max_length=max_length, truncation=True)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)

    # Creating attention mask
    attention_mask = torch.zeros_like(tokens_tensor).long()
    attention_mask[0, :len(indexed_tokens)] = 1

    # Performing the inference
    with torch.no_grad():
        outputs = model(tokens_tensor, attention_mask=attention_mask)
        logits = outputs[0] if len(outputs) == 1 else outputs[1]
        _, predicted = torch.max(logits, dim=1)
        relation_id = predicted.item()

    # Output the predicted relation
    predicted_relation = id2rel[relation_id]

    return predicted_relation

In [18]:
# input what you want
text = "He has served as a policy aide to the late U.S. Senator Alan Cranston , as National Issues Director for the 2004 presidential campaign of Congressman Dennis Kucinich , as a co-founder of Progressive Democrats of America and as a member of the international policy department at the RAND Corporation think tank before all that ."
ent1 = "Progressive Democrats of America"
ent2 = "international policy department"

In [19]:
# Demo predicted Relation
model_path = "/content/Drive/MyDrive/model/BERT_EX1_100_model.pth"
predicted_relation = predict_relation("BERT", ent1, ent2, text, model_path, rel2id, id2rel)
print(f"Predicted Relation: {predicted_relation}")

Predicted Relation: no_relation
